In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd "/content/drive/MyDrive/Colab Notebooks/SemEval2023/Evaluation"

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install colorama
!pip install wikipedia-api
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sys
import os
sys.path.append('../')
import pandas as pd
import torch 
import numpy as np
from tqdm import tqdm
import random
from datasets import Dataset
import nltk
nltk.download('punkt')
os.environ["WANDB_DISABLED"] = "true"
from helper import prepare_data


[nltk_data] Downloading package punkt to /home/hossain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Seed all

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

/home/hossain/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
LANG = 'fr' # use None for all lang
SET = 'LM' # 'LM' or None
EVAL_SET = 'test'

In [6]:
if LANG=='en' and SET=='LM':
    !python -m spacy download en_core_web_lg

## Read Data

In [7]:
filename = f'../Dataset/{LANG}-{EVAL_SET}.conll'
data = prepare_data(filename)

In [8]:
data.head()

,ID,lang,sent,labels
0,b09e9283-4dd5-464b-b395-a0ad0cb8447d,None,lola marois bigard ( écrivain actrice et...,
1,f57e302f-5bb2-40a4-8b72-882e7d1747f0,None,le film manhattan de woody allen comport...,
2,3717969f-16b1-4e36-9e9f-c09fb27a35ab,None,1994 : little big league d andrew schei...,
3,775659c2-d06f-4b64-9672-ccbd023f354b,None,il y devient célèbre et même « inoublia...,
4,59312052-ade0-41d8-a35a-0af6aee98d4e,None,de 1975 à 1977 elle a une histoire d ...,


## Augment Info

In [9]:
from InformationExtraction import InformationExtractionPipeline
infoPipeline = InformationExtractionPipeline(SET,
                                        max_sen = 2, lang = LANG, 
                                        loadJson = True, jsonPath=f'./Wiki/{LANG}-wiki.json',
                                            saveJson=True, saveJsonpath = f'./Wiki/{LANG}-test-wiki.json')

2023-01-27 17:41:07.289875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 17:41:08.390058: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2023-01-27 17:41:08.390119: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2023-01-27 17:41:08.390124: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [ ]:
augmented = infoPipeline(data[['sent','labels']].values.tolist())


  0%|                                   | 470/249786 [04:09<36:08:30,  1.92it/s]

HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


  1%|▍                                 | 3647/249786 [23:23<29:15:51,  2.34it/s]

Expecting value: line 1 column 1 (char 0)


  2%|▌                                 | 4325/249786 [28:50<19:13:28,  3.55it/s]

...

 19%|█████▊                         | 47061/249786 [3:14:35<13:54:50,  4.05it/s]

HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 24%|███████▊                        | 60696/249786 [3:45:46<4:46:01, 11.02it/s]

HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 29%|████████▊                      | 71270/249786 [4:30:42<14:22:31,  3.45it/s]

In [ ]:
data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

In [ ]:
dir = f'./Augmented-Dataset'
if not os.path.exists(dir):
    os.mkdir(dir)
data.to_csv(f'{dir}/{LANG}-test.csv',index=False)

In [ ]:
from datasets import load_dataset, Dataset

data = Dataset.from_pandas(data)

In [ ]:
data